<a href="https://colab.research.google.com/github/A-Elakad/Organic-Grocery-Association-Rules/blob/main/Organic_Grocery_Association_Rules_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform

In [ ]:
main_frame = pd.read_csv('/content/sales.csv')

In [ ]:
main_frame['product_name'].nunique()

In [ ]:
main_frame['product_id'].nunique()

In [ ]:
main_frame['product_name'].isnull().sum()

In [ ]:
df = main_frame.drop(['order_delivery_cost','product_price','order_total_products_price','product_stock','order_total_price','order_customer_type','order_date'],axis=1)
df

In [ ]:
df.isnull().sum()

In [ ]:
# Replace 'column_name' with the name of the column you want to check
rows_with_null = df[df['product_name'].isnull()]

# Display the rows
rows_with_null['product_id'].unique()

In [ ]:
# Assuming 'column_name' is the column where you want to check the occurrences
values_to_check = [
    '65c69a336fbd3f330902e522', '65c6ad2db41824de3a0e810a',
    '65c6ab58b1cef436b60fc512', '65c6b0018b0b17cfd90c4f45',
    '65c6b2de3b0f70dd0d0721f5', '65c773dcbdf63d214e0bce44',
    '65c6ac532d470f225101de13', '65c69a3f52f83b6f7307a884',
    '65c693a3e437225e2d079122', '65c6b345c1ea7c9ce70e5488',
    '65c69be7434f321680034873', '65c695d3c38dda787c034042',
    '65c69c32434f32168003487c', '65c6b18508ca2a15a500b522',
    '664a0b0324c765989b0dc354'
]

# Filter the DataFrame to include only rows where the column contains values from the array
filtered_df = df[df['product_id'].isin(values_to_check)]

# Count occurrences of each value in the filtered DataFrame
value_counts = filtered_df['product_id'].value_counts()

# Display the result
print(value_counts)


In [ ]:
# Create a dictionary to map product names to encoded values
product_encoder = {product_name: idx for idx, product_name in enumerate(df['product_name'].unique())}

# Create a reverse dictionary to decode encoded values back to product names
product_decoder = {idx: product_name for product_name, idx in product_encoder.items()}

In [ ]:
# Apply encoding to the 'product_id' column in the original DataFrame
df['product_name_encoded'] = df['product_name'].map(product_encoder)

In [ ]:
# Group orders and perform one-hot encoding (assuming 'order_id' is your order identifier column)
grouped_orders = df.groupby('order_id')['product_name_encoded'].apply(list).reset_index()
te = TransactionEncoder()
te_ary = te.fit(grouped_orders['product_name_encoded']).transform(grouped_orders['product_name_encoded'])
df_new = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
order_sizes = df.groupby('order_id')['product_name_encoded'].count()

plt.hist(order_sizes, bins=15)
plt.xlabel('Number of Products per Order')
plt.ylabel('Frequency')
_ = plt.title('Distribution of order sizes')

In [ ]:
frequent_itemsets = apriori(df_new, min_support=0.008, use_colnames=True)
#Extract the itemsets as lists from the sets
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(list)

# One-hot encode the itemsets
mlb = MultiLabelBinarizer()
itemsets_encoded = mlb.fit_transform(frequent_itemsets['itemsets'])

# Create a new DataFrame with one-hot encoded columns
itemsets_encoded_df = pd.DataFrame(itemsets_encoded, columns=mlb.classes_)

print(frequent_itemsets)

In [ ]:
# Modify the decode_itemsets function to handle lists of itemsets
def decode_itemsets(itemsets):
    # Decode each item in the list of itemsets
    return {product_decoder[item] for item in itemsets}

# Apply the modified function to decode the itemsets
frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(decode_itemsets)

# Print the decoded itemsets
print(frequent_itemsets)

In [ ]:
frequent_itemsets.to_excel('frequent_itemsets (Organic).xlsx', index=False)

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.75)

rules

In [ ]:
# Define a function to decode a frozenset of product IDs
def decode_itemset(itemset, product_decoder):
  return {product_decoder[item] for item in itemset}

# Assuming 'antecedents' and 'consequents' are already in a format that decode_itemset can handle
# Decode the 'antecedents' and 'consequents' columns using the product_decoder
rules['antecedents_decoded'] = rules['antecedents'].apply(lambda x: decode_itemset(x, product_decoder))
rules['consequents_decoded'] = rules['consequents'].apply(lambda x: decode_itemset(x, product_decoder))

# Reorder the columns to place 'antecedents_decoded' as the second column and 'consequents_decoded' as the fourth column
cols = rules.columns.tolist()

# Insert 'antecedents_decoded' after 'antecedents' (second column) and 'consequents_decoded' after 'consequents' (fourth column)
cols.insert(1, cols.pop(cols.index('antecedents_decoded')))  # Move 'antecedents_decoded' to second column
cols.insert(3, cols.pop(cols.index('consequents_decoded')))  # Move 'consequents_decoded' to fourth column

# Reorder the DataFrame based on the new column order
rules = rules[cols]

# Display the updated DataFrame to verify
print(rules.head())

In [ ]:
rules.to_excel('association_rules (Organic).xlsx', index=False)